In [1]:
import os
os.chdir('E:tasawar\ML_Labs')

In [2]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
#from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
#from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate, Dense
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
from keras.callbacks import Callback

In [3]:
#lookback = 24
model = None
start_epoch = 0
time_steps=24
num_features=21

In [4]:
def CNN():
    input_data = Input(shape=(time_steps, num_features))
    x1 = Conv1D(16, 2, activation="relu")(input_data)
    x2 = Conv1D(16, 2, activation="relu")(x1)
    flatten = Flatten()(x2)
    output_data = Dense(1)(flatten)
    model = Model(input_data, output_data)
    return model

In [5]:
model1 = CNN()
model1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 24, 21)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 23, 16)         │           688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 22, 16)         │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 352)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           353 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,569 (6.13 KB)

 Trainable params: 1,569 (6.13 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
tensorflow.keras.utils.plot_model(model1 )

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [7]:
checkpoints = r'E:tasawar\ML_Labs\venv\Lab_8\\E1-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'E:tasawar\ML_Labs\venv\Lab_8'
#FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
#JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])

In [8]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
#TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1]

In [9]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model =CNN()
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


In [20]:
import os
path_dataset =r'E:tasawar\ML_Labs\venv\Lab_8'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(r'E:tasawar\ML_Labs\venv\AEP_train.csv')
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(r'E:tasawar\ML_Labs\venv\AEP_validation.csv')
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(r'E:tasawar\ML_Labs\venv\AEP_test.csv')
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler      = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

e:\ML_Labs\venv\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


((84907, 21), (24259, 21), (12130, 21))

In [21]:
time_steps=24
num_features=21

In [22]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=1)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=1)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=1)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 1.0484118461608887 sec


In [23]:
epochs = 10
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,verbose = verbose)

Epoch 1/10
2649/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0668 - mae: 0.0668 - mape: 165.1008
Epoch 1: val_loss improved from inf to 0.02981, saving model to E:\ML_Labs\venv\Lab_8\\E1-cp-0001-loss0.03.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 23s 7ms/step - loss: 0.0667 - mae: 0.0667 - mape: 167.6283 - val_loss: 0.0298 - val_mae: 0.0298 - val_mape: 14.5882
Epoch 2/10
2650/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0250 - mae: 0.0250 - mape: 1894.6309
Epoch 2: val_loss improved from 0.02981 to 0.02086, saving model to E:\ML_Labs\venv\Lab_8\\E1-cp-0002-loss0.02.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 15s 5ms/step - loss: 0.0250 - mae: 0.0250 - mape: 1893.6384 - val_loss: 0.0209 - val_mae: 0.0209 - val_mape: 10.3751
Epoch 3/10
2645/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0185 - mae: 0.0185 - mape: 2054.7141
Epoch 3: val_loss improved from 0.02086 to 0.01594, saving model to E:\ML_Labs\venv\Lab_8\\E1-cp-0003-loss0.02.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - loss: 0.0185 - mae: 0.0185 - mape: 2050.9248 - val_loss: 0.0159 - val_mae: 0.0159 - val_mape: 7.7607
Epoch 4/10
2645/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0146 - mae: 0.0146 - mape: 147.3192
Epoch 4: val_loss improved from 0.01594 to 0.01378, saving model to E:\ML_Labs\venv\Lab_8\\E1-cp-0004-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 16s 6ms/step - loss: 0.0146 - mae: 0.0146 - mape: 148.0643 - val_loss: 0.0138 - val_mae: 0.0138 - val_mape: 7.2342
Epoch 5/10
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0126 - mae: 0.0126 - mape: 2433.6235
Epoch 5: val_loss improved from 0.01378 to 0.01192, saving model to E:\ML_Labs\venv\Lab_8\\E1-cp-0005-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.0126 - mae: 0.0126 - mape: 2432.9131 - val_loss: 0.0119 - val_mae: 0.0119 - val_mape: 5.6967
Epoch 6/10
2645/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0115 - mae: 0.0115 - mape: 490.9499
Epoch 6: val_loss improved from 0.01192 to 0.01174, saving model to E:\ML_Labs\venv\Lab_8\\E1-cp-0006-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - loss: 0.0115 - mae: 0.0115 - mape: 490.8419 - val_loss: 0.0117 - val_mae: 0.0117 - val_mape: 6.2431
Epoch 7/10
2647/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0107 - mae: 0.0107 - mape: 110.4140
Epoch 7: val_loss improved from 0.01174 to 0.00977, saving model to E:\ML_Labs\venv\Lab_8\\E1-cp-0007-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - loss: 0.0107 - mae: 0.0107 - mape: 111.3113 - val_loss: 0.0098 - val_mae: 0.0098 - val_mape: 5.3238
Epoch 8/10
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0104 - mae: 0.0104 - mape: 175.7878
Epoch 8: val_loss did not improve from 0.00977
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - loss: 0.0104 - mae: 0.0104 - mape: 176.0917 - val_loss: 0.0103 - val_mae: 0.0103 - val_mape: 5.1870
Epoch 9/10
2643/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0099 - mae: 0.0099 - mape: 1005.0353
Epoch 9: val_loss did not improve from 0.00977
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.0099 - mae: 0.0099 - mape: 1003.4932 - val_loss: 0.0100 - val_mae: 0.0100 - val_mape: 4.9516
Epoch 10/10
2646/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0096 - mae: 0.0096 - mape: 289.0870
Epoch 10: val_loss improved from 0.00977 to 0.00927, saving model to E:\ML_Labs\venv\Lab_8\\E1-cp-0010-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - loss: 0.0096 - mae: 0.0096 - mape: 290.2037 - val_loss: 0.0093 - val_mae: 0.0093 - val_mape: 5.0241


In [27]:
model = load_model(r'E:tasawar\ML_Labs\venv\Lab_8\\E1-cp-0002-loss0.02.h5', compile=False)

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step
Mean Absolute Error (MAE): 342.03
Median Absolute Error (MedAE): 292.61
Mean Squared Error (MSE): 183610.27
Root Mean Squared Error (RMSE): 428.5
Mean Absolute Percentage Error (MAPE): 2.39 %
Median Absolute Percentage Error (MDAPE): 2.0 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


In [28]:
checkpoints =r'E:tasawar\ML_Labs\venv\Lab_8\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=r'E:\ML_Labs\venv\Lab_8\E1-cp-0058-loss0.01.h5'
start_epoch= 58

In [32]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

if model is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(1e-3)
    model.compile(loss='mae', optimizer=opt, metrics=["mae", "mape"])
else:
    print("[INFO] loading model from disk...")
    model = load_model(r'E:\ML_Labs\venv\Lab_8\\E1-cp-0002-loss0.02.h5', compile=False)

    # Manually compile since we used compile=False
    opt = Adam(1e-3)
    model.compile(loss='mae', optimizer=opt, metrics=["mae", "mape"])

    # Print and update learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.learning_rate)))
    model.optimizer.learning_rate = 1e-4

    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.learning_rate)))


[INFO] loading model from disk...
[INFO] old learning rate: 0.0010000000474974513
[INFO] new learning rate: 9.999999747378752e-05


In [33]:
epochs = 5
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

Epoch 1/5
2645/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0160 - mae: 0.0160 - mape: 285.4297
Epoch 1: val_loss improved from inf to 0.01615, saving model to E:\ML_Labs\venv\Lab_8\E2-cp-0001-loss0.02.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 28s 9ms/step - loss: 0.0160 - mae: 0.0160 - mape: 287.0661 - val_loss: 0.0162 - val_mae: 0.0162 - val_mape: 7.9955
Epoch 2/5
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0150 - mae: 0.0150 - mape: 769.1888
Epoch 2: val_loss improved from 0.01615 to 0.01552, saving model to E:\ML_Labs\venv\Lab_8\E2-cp-0002-loss0.02.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0150 - mae: 0.0150 - mape: 769.2213 - val_loss: 0.0155 - val_mae: 0.0155 - val_mape: 7.6715
Epoch 3/5
2648/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0144 - mae: 0.0144 - mape: 377.0912
Epoch 3: val_loss improved from 0.01552 to 0.01513, saving model to E:\ML_Labs\venv\Lab_8\E2-cp-0003-loss0.02.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - loss: 0.0144 - mae: 0.0144 - mape: 377.5200 - val_loss: 0.0151 - val_mae: 0.0151 - val_mape: 7.4277
Epoch 4/5
2644/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0140 - mae: 0.0140 - mape: 785.6876
Epoch 4: val_loss improved from 0.01513 to 0.01458, saving model to E:\ML_Labs\venv\Lab_8\E2-cp-0004-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 0.0140 - mae: 0.0140 - mape: 784.6270 - val_loss: 0.0146 - val_mae: 0.0146 - val_mape: 7.1505
Epoch 5/5
2645/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0135 - mae: 0.0135 - mape: 331.6324
Epoch 5: val_loss improved from 0.01458 to 0.01416, saving model to E:\ML_Labs\venv\Lab_8\E2-cp-0005-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0135 - mae: 0.0135 - mape: 331.8683 - val_loss: 0.0142 - val_mae: 0.0142 - val_mape: 6.7002


In [34]:

model =  load_model(r'E:tasawar\ML_Labs\venv\Lab_8\\E1-cp-0002-loss0.02.h5', compile=False)

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Mean Absolute Error (MAE): 342.03
Median Absolute Error (MedAE): 292.61
Mean Squared Error (MSE): 183610.27
Root Mean Squared Error (RMSE): 428.5
Mean Absolute Percentage Error (MAPE): 2.39 %
Median Absolute Percentage Error (MDAPE): 2.0 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)
